In [2]:
import pandas as pd
import numpy as np
import re
from estnltk import Text
import sqlite3

Verbide ja nende mustrite eraldamine ja kergemini kättesaadavale kujule viimine. Tulemuseks:

    Üheliikmelised mustrid - verb + üks fraas; nt aasima + sõbra kallal
    Kaheliikmelised mustrid - verb + kaks fraasi; nt küsima + sõbra käest + raamatut
    
Mustrid on esitatud kujul, kus on eraldi tulpades verb, verbi muu(d) osa(d), kääne, kaassõna, infiniitverb või muusse kategooriasse kuuluv liige.
    

In [3]:
# algne exceli fail rektsiooniinfoga
df = pd.read_excel('data/verb_pattern_data/12_eesti_ilmik_rektsiooniga_linkideta.xlsx')

In [4]:
df

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government
0,154548,1141002,aasima,1,True,v,keda*
1,154548,1141002,aasima,1,True,v,kelle kallal
2,154678,1141297,abielluma,1,True,v,kellega
3,154740,1141437,abikätt ulatama,1,True,v,kellele
4,154772,1141513,abistama,1,True,v,keda/mida* + milles
...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega
3320,264058,1413464,ümmardama,1,True,v,keda*
3321,264120,1413609,üritama,1,True,v,mida teha
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta


### Mustrite eraldamine

#### I Mappingud küsimuste-käänete jaoks

In [216]:
# mapping 14 + 1 cases to their corresponding questions

#ab	abessiiv (ilmaütlev)
#abl	ablatiiv (alaltütlev)
#ad	adessiiv (alalütlev)
#adt	aditiiv suunduv (lühike sisseütlev)
#all	allatiiv (alaleütlev)
#el	elatiiv (seestütlev)
#es	essiiv (olev)
#g	genitiiv (omastav)
#ill	illatiiv (sisseütlev)
#in	inessiiv (seesütlev)
#kom	komitatiiv (kaasaütlev)
#n	nominatiiv (nimetav)
#p	partitiiv (osastav)
#ter	terminatiiv (rajav)
#tr	translatiiv (saav)

# käänded-küsimused
case_question_mapping = {'nom': ['kes', 'mis', 'milline'],
                   'gen': ['kelle', 'mille', 'millise'],
                   'part': ['keda', 'mida', 'millist'],
                   'ill': ['kellesse', 'millesse', 'millisesse'],
                   'adit': ['kuhu'],
                   'in': ['kelles', 'milles', 'kus', 'millises'],
                   'el': ['kellest', 'millest', 'kust', 'millisest'],
                   'all': ['kellele', 'millele', 'kuhu', 'millisele'],
                   'ad': ['kellel', 'millel', 'kus', 'millisel'],
                   'abl': ['kellelt', 'millelt', 'kust', 'milliselt'],
                   'tr': ['kelleks', 'milleks', 'milliseks'],
                   'term': ['kelleni', 'milleni', 'milliseni'],
                   'es': ['kellena', 'millena', 'millisena'],
                   'abes': ['kelleta', 'milleta', 'milliseta'],
                   'kom': ['kellega', 'millega', 'millisega']}


# küsimused-käänded
question_case_mapping = {'kes': 'nom', 'mis': 'nom', 'milline': 'nom',
                   'kelle': 'gen', 'mille': 'gen', 'millise': 'gen',
                   'keda': 'part', 'mida': 'part', 'millist': 'part',
                   'kellesse': 'ill', 'millesse': 'ill', 'millisesse': 'ill',
                   'kuhu': 'adit',
                   'kelles': 'in', 'milles': 'in', 'kus': 'in', 'millises': 'in',
                   'kellest': 'el', 'millest': 'el', 'kust': 'el', 'millisest': 'el',
                   'kellele': 'all', 'millele': 'all', 'kuhu': 'all', 'millisele': 'all',
                   'kellel': 'ad', 'millel': 'ad', 'kus': 'ad', 'millisel': 'ad',
                   'kellelt': 'abl', 'millelt': 'abl', 'kust': 'abl', 'milliselt': 'abl',
                   'kelleks': 'tr', 'milleks': 'tr', 'milliseks': 'tr',
                   'kelleni': 'term', 'milleni': 'term', 'milliseni': 'term',
                   'kellena': 'es', 'millena': 'es', 'millisena': 'es',
                   'kelleta': 'abes', 'milleta': 'abes', 'milliseta': 'abes',
                   'kellega': 'kom', 'millega': 'kom', 'millisega': 'kom'}

# NB! Hetkel pole veel lahendatud ära küsisõna "kuhu" mitmesust (mis käänetena võime seda tõlgendada),
#                                                                        sellega tegeletakse hiljem.
#     Mappingus on 'kuhu' märgitud küll nii lühikeseks sisseütlevaks kui alaleütlevaks,
#     aga tulemusena saadud mustrites on hetkel kõik 'kuhu' juhud märgitud alaleütlevaks käändeks (all)

In [217]:
# Tähelepanekud veeru 'government' kohta algses exceli failis:
#
# -- '/' tähistab üht või teist ehk alternatiive, nt käänete puhul elus/eluta või siis alternatiivseid mustreid
# -- '*' tähendust ei tea
# -- ' + ' käib mitmest fraasist koosneva mustri liikmete vahele 
# --      (nt kääne + kaassõna, kääne koos kaassõnaga + kääne, kääne + kääne vms)
# -- alati pole käändeinfot, kuigi enamasti on. Vahel esineb verbiga koos lihtsalt kaassõna, sidesõna vms
# -- tühikuga on eraldatud ühe terviku/fraasi osad
#
# Tähelepanekud veeru 'pos' kohta:
#
# -- valdavalt verbid
# -- kohati s, adv, adj, prep, postp, vrm
# -- korraga võimalik rohkem, kui üks väärtus
# -- kohati on lahter tühi. Tavaliselt on siiski eelmises/järgnevas reas identne näide.
#
#
# Lähenemine:
#
# -- võtta esiotsa kõik read, ignoreerides sõnaliiki
# -- eraldada küsisõna ja muud liikmed, küsisõna puudumisel ainult muu liige. Küsisõna asemele läheb kääne
# -- hetkel ignoreerida tärni (*)
# -- alternatiivsed mustrid lüüa lahku eraldi ridadele (eraldi mustriteks)
# -- eraldada üksteisest üheliikmelised ja kaheliikmelised mustrid
# -- lüüa lahku verb, selle osised (compound_prt) ning mustri osised kääne, kaassõna, infiniitverb või 
# --                                                                   muusse kategooriasse kuuluv osa
# -- salvestada andmebaasi eraldi tabelitesse nii ühe- kui kaheliikmelised mustrid

In [219]:
# test kombinatsioonid
test1 = 'kelle käest + mida*'
test2 = 'kelle/mille kohta'
test3 = 'kelle/mida kohta'
test4 = 'millele / mille peale'
test5 = 'mis'

#### II Verbiühendite tükeldamine

In [220]:
# verbide tükeldamine (pea)verbiks ja selle muudeks osadeks
verb_word = []
compound_prt1 = []
compound_prt2 = []
compound_prt3 = []

for idx, row in df.iterrows():
    verb = ''
    compound = ['', '', ''] # in current data, there are max 3 compound pieces
    pieces = row['word'].split()
    
    j = len(pieces)-1 # (pea)verbi leidmine. Oletus on, et pikema konstruktsiooni viimane verbist liige on peaverb
    while j >= 0:
        text = Text(pieces[j]).tag_layer('morph_analysis')
        if 'V' in text.morph_analysis.partofspeech[0]:
            verb = pieces[j]
            pieces.pop(j)
            break
        j-=1
        
    for i in range(len(pieces)): # allesjäänud jupid määratakse konstruktsiooni ülejäänud osadeks
        compound[i] = pieces[i]
    
    verb_word.append(verb)
    compound_prt1.append(compound[0])
    compound_prt2.append(compound[1])
    compound_prt3.append(compound[2])
    
df['verb_word'] = verb_word
df['compound_prt1'] = compound_prt1
df['compound_prt2'] = compound_prt2
df['compound_prt3'] = compound_prt3

In [221]:
df

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government,verb_word,compound_prt1,compound_prt2,compound_prt3
0,154548,1141002,aasima,1,True,v,keda*,aasima,,,
1,154548,1141002,aasima,1,True,v,kelle kallal,aasima,,,
2,154678,1141297,abielluma,1,True,v,kellega,abielluma,,,
3,154740,1141437,abikätt ulatama,1,True,v,kellele,ulatama,abikätt,,
4,154772,1141513,abistama,1,True,v,keda/mida* + milles,abistama,,,
...,...,...,...,...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega,ümbritsema,,,
3320,264058,1413464,ümmardama,1,True,v,keda*,ümmardama,,,
3321,264120,1413609,üritama,1,True,v,mida teha,üritama,,,
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta,ütlema,,,


#### III Puhastus ning küsisõnade asendamine käänetega

In [222]:
def get_mapping(text, mapping_dict):
    if text in mapping_dict.keys():
        return question_case_mapping[text]
    return text

def split_members(text):
    cleaned_text = re.sub('\*', '', text) # tärnide eemaldamine
    splitted = cleaned_text.split(' + ') # liikmete tükeldamine
    
    for i in range(len(splitted)):
        splitted[i] = splitted[i].split() # liikme alamosade tükeldamine tühiku kohalt         
        
        for j in range(len(splitted[i])):
            if '/' in splitted[i][j] and splitted[i][j] != '/':
                splitted[i][j] = splitted[i][j].split('/') # alternatiivide tükeldamine
            
                for k in range(len(splitted[i][j])): # küsisõnade asendamine sobivate käänetega
                    splitted[i][j][k] = get_mapping(splitted[i][j][k], question_case_mapping)
                    
                splitted[i][j] = set(splitted[i][j]) # kui alternatiivideks on tegelikult sama kääne, siis eemaldatakse duplikaadid
                splitted[i][j] = ' '.join(splitted[i][j]) # NB! Hetkel veel ei eraldata alternatiivseid mustreid
                
            else:
                splitted[i][j] = get_mapping(splitted[i][j], question_case_mapping)

        splitted[i] = ' '.join(splitted[i])
        
    return ' + '.join(splitted)

In [223]:
print(split_members(test1))
print(split_members(test2))
print(split_members(test3))
print(split_members(test4))
print(split_members(test5))

gen käest + part
gen kohta
part gen kohta
all / gen peale
nom


In [224]:
# Tulemus:
#
# -- ' + ' eraldab pikema konstruktsiooni eri liikmeid
# -- ' / ' eraldab eri võimalikke variante
# -- ' ' käib sama liikme eri juppide (sõnade) vahele (nt kääne koos kaassõnaga)
#
# NB! Esialgu lisan käänetega mustrid kõigile rektsioonitabeli ridadele, sõltumata sõnaliigist

In [225]:
# veeru lisamine, kus on government, aga küsisõna asemel on nüüd kääne
patterns_with_case = []

for idx, row in df.iterrows():
    patterns_with_case.append(split_members(row['government']))

In [226]:
len(patterns_with_case)

3324

In [227]:
df['government_with_case'] = patterns_with_case

In [228]:
df

,wordid,lexeme_id,word,homonym_nr,is_public,pos,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case
0,154548,1141002,aasima,1,True,v,keda*,aasima,,,,part
1,154548,1141002,aasima,1,True,v,kelle kallal,aasima,,,,gen kallal
2,154678,1141297,abielluma,1,True,v,kellega,abielluma,,,,kom
3,154740,1141437,abikätt ulatama,1,True,v,kellele,ulatama,abikätt,,,all
4,154772,1141513,abistama,1,True,v,keda/mida* + milles,abistama,,,,part + in
...,...,...,...,...,...,...,...,...,...,...,...,...
3319,264050,1413420,ümbritsema,1,True,v,millega,ümbritsema,,,,kom
3320,264058,1413464,ümmardama,1,True,v,keda*,ümmardama,,,,part
3321,264120,1413609,üritama,1,True,v,mida teha,üritama,,,,part teha
3322,264132,1413655,ütlema,1,True,v,kelle/mille kohta,ütlema,,,,gen kohta


In [229]:
# vahetulemuse salvestamine
df.to_csv('data/verb_pattern_data/rektsioon_kaanetega.csv', encoding='utf-8')

#### IV Ühe- ja kaheliikmeliste ning alternatiivsete mustrite eraldamine

In [230]:
df = pd.read_csv('data/verb_pattern_data/rektsioon_kaanetega.csv')

In [231]:
# edaspidi huvipakkuvad veerud kopeeritakse
df3 = df[['word', 'government', 'verb_word', 'compound_prt1', 'compound_prt2', 'compound_prt3', 'government_with_case']].copy()
df3

,word,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case
0,aasima,keda*,aasima,NaN,NaN,NaN,part
1,aasima,kelle kallal,aasima,NaN,NaN,NaN,gen kallal
2,abielluma,kellega,abielluma,NaN,NaN,NaN,kom
3,abikätt ulatama,kellele,ulatama,abikätt,NaN,NaN,all
4,abistama,keda/mida* + milles,abistama,NaN,NaN,NaN,part + in
...,...,...,...,...,...,...,...
3319,ümbritsema,millega,ümbritsema,NaN,NaN,NaN,kom
3320,ümmardama,keda*,ümmardama,NaN,NaN,NaN,part
3321,üritama,mida teha,üritama,NaN,NaN,NaN,part teha
3322,ütlema,kelle/mille kohta,ütlema,NaN,NaN,NaN,gen kohta


In [232]:
# kaheliikmelised mustrid
for idx, row in df3.iterrows():
    if '+' in row['government_with_case']:
        print(row['government_with_case'])

part + in
ad + part teha
all + part
part + el
part + kom
ad + part teha
all + gen peale
part + kuidas
kom + in
ad + part teha
gen käest + part
abl + part
ad + part teha
ad + part teha
ad + part teha
kom + gen suhtes
ad + tr
ad + part teha
part + tr
part + tr
all + el
all + part
ad + part teha
part + part tegema
ad + part teha
part + el
abl + all
all + kom
ad + part teha
nom + es
nom keelest + nom keelde
part + kom
all + kom
abl + all
ad + part teha


In [233]:
# alternatiivsed mustrid
count = 0
for idx, row in df3.iterrows():
    if '/' in row['government_with_case']:
        print(row['government_with_case'])
        count += 1
print(count)

abl / gen käest
tr / part teha
el / gen kohta
gen juurde / gen kallale
gen tõttu / gen tagajärjel
gen pärast / gen tõttu
gen peale / gen eest
all / gen peale
el / gen eest
abl / el
all / gen peale
gen eest / kom
ad / in
gen eest / kom
abl / gen käest
abl / abl
vastu part / gen vastu
el / gen hulgast
kom / gen vastu
ad / gen juures
20


In [234]:
# uus koopia, et oleks võimalik indeksite alusel ridu lisada
koopia = df3.copy()

In [236]:
# alternatiivsed mustrid lüüakse lahku ning sisestatakse dataframe-i eraldi ridadele
i = 0
size = len(koopia)
pattern = r'(\D+) / (\D+)'
while i < size-1:
    if '/' in koopia.loc[i]['government_with_case']:
        row_to_add = koopia.loc[i].copy()
        row_to_add['government_with_case'] = re.sub(pattern, r'\2', koopia.loc[i]['government_with_case'])
        koopia.loc[i]['government_with_case'] = re.sub(pattern, r'\1', koopia.loc[i]['government_with_case'])
        print(koopia.loc[i]['government_with_case'])
        print(row_to_add['government_with_case'])
        print()
        koopia.loc[(i+1)/i] = row_to_add
        koopia = koopia.sort_index().reset_index(drop=True)
        i+=2
        size=len(koopia)
    else:
        i+=1

abl
gen käest

tr
part teha

el
gen kohta

gen juurde
gen kallale

gen tõttu
gen tagajärjel

gen pärast
gen tõttu

gen peale
gen eest

all
gen peale

el
gen eest

abl
el

all
gen peale

gen eest
kom

ad
in

gen eest
kom

abl
gen käest

abl
abl

vastu part
gen vastu

el
gen hulgast

kom
gen vastu

ad
gen juures



In [237]:
# kontroll, ega pole jäänud mõni alternatiiv lahku löömata
count = 0
for idx, row in koopia.iterrows():
    if '/' in row['government_with_case']:
        print(row['government_with_case'])
        count += 1
print(count)

0


In [238]:
# indeksite reset
koopia = koopia.sort_values('word').reset_index(drop=True)

In [239]:
# korduvad read eemaldatakse
koopia = koopia.drop_duplicates(subset = ['verb_word', 'compound_prt1', 'compound_prt2', 'compound_prt3', 'government_with_case'])

In [240]:
# uuesti indeksite reset
koopia = koopia.reset_index(drop=True)

In [241]:
koopia

,word,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case
0,aasima,keda*,aasima,NaN,NaN,NaN,part
1,aasima,kelle kallal,aasima,NaN,NaN,NaN,gen kallal
2,abielluma,kellega,abielluma,NaN,NaN,NaN,kom
3,abikätt ulatama,kellele,ulatama,abikätt,NaN,NaN,all
4,abistama,keda/mida* + milles,abistama,NaN,NaN,NaN,part + in
...,...,...,...,...,...,...,...
2566,ümmardama,keda*,ümmardama,NaN,NaN,NaN,part
2567,üritama,mida teha,üritama,NaN,NaN,NaN,part teha
2568,ütlema,kellele,ütlema,NaN,NaN,NaN,all
2569,ütlema,kelle/mille kohta,ütlema,NaN,NaN,NaN,gen kohta


In [242]:
# mustris üks liige, mis koosneb ühest sõnast, kuid mis ei ole kääne: põhiliselt sidesõnad, aja- või viisimäärused
# üks kord esinevad erandid: küsisõna 'kas', venekeelne sõna 'tšto' - 'mis', kaasaütlev kääne koos gi-liitega 'kellegagi' 
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) == 1:
        has_no_case = True
        for piece in row['government_with_case'].split():
            if piece in case_question_mapping.keys():
                has_no_case = False
        if has_no_case:
            print(row['word'], row['government_with_case'].split())

arvama ['et']
deklareerima ['et']
demonstreerima ['et']
eeldama ['et']
ette kujutama ['kuidas']
ette kujutama ['et']
hakkama ['millal']
hoolitsema ['et']
hoolt kandma ['et']
kaebama ['et']
kahetsema ['et']
kahtlema ['kas']
kahtlustama ['et']
kartma ['et']
kehtestama ['millal']
kinnitama ['et']
kogema ['et']
kokku leppima ['et']
kordama ['et']
kulgema ['kuidas']
kurtma ['et']
kuulma ['et']
käituma ['kuidas']
küsima ['kas']
leidma ['et']
lootma ['et']
märkama ['et']
märkima ['et']
näima ['et']
näitama ['et']
oletama ['et']
otsustama ['et']
pahandama ['et']
paistma ['et']
peale hakkama ['millal']
rääkima ['et']
rõhutama ['et']
süüdistama ['et']
tajuma ['et']
teatama ['et']
teesklema ['et']
toimima ['kuidas']
tundma ['kuidas']
tunduma ['et']
tunnetama ['et']
tunnistama ['et']
tuvastama ['et']
tähele panema ['et']
tähendama ['et']
täpsustama ['kas']
tõdema ['et']
tõestama ['et']
unistama ['et']
uskuma ['et']
vabandama ['et']
valima ['kas']
veenduma ['et']
vihjama ['et']
väitma ['et']
võtma 

In [243]:
# eraldatakse muud erandlikud juhud: juhud, kus ühene muster koosneb mitmest sõnast, mille seas puudub kääne
# VÕI koosneb ühene muster rohkem, kui ühest käändest. Sel juhul tõlgendatakse seda kaheliikmelise mustrina
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) > 1 and '+' not in row['government_with_case'].split():
        has_no_case = True
        n_cases = 0
        for piece in row['government_with_case'].split():
            if piece in case_question_mapping.keys():
                has_no_case = False
                n_cases+=1
        if has_no_case:
            print(row['word'], row['government_with_case'].split())
        elif n_cases > 1:
            print(row['word'], row['government_with_case'].split())

alla kirjutama ['part', 'gen']
kaasa lööma ['in', 'ad']
kaasa tegema ['in', 'ad']
kannatama ['gen', 'all']
kestma ['kui', 'kaua']
kõrvale jääma ['abl', 'el']
mugandama ['kom', 'all']
nimetama ['part', 'tr']
näima ['nom', 'es']
ostma ['part', 'gen']
teada saama ['el', 'gen', 'kohta']
tuhnima ['в', 'чём']
ära harjutama ['ad', 'part']
ära harjutama ['part', 'el']


In [244]:
# erandlik juht, kus 'üheliikmeline' koosneb rohkem, kui kahest sõnast. Edaspidi käsitleme kaheliikmelise mustrina
for idx, row in koopia.iterrows():
    if len(row['government_with_case'].split()) > 2 and '+' not in row['government_with_case']:
        print(row['word'], row['government_with_case'].split())

teada saama ['el', 'gen', 'kohta']


#### V Mustrite tükeldamine

In [248]:
# kui verbi mustrid on üks liige (võib koosneda rohkem, kui ühest sõnast (siinsetes andmetes kuni 3 sõnast), 
# kuid need sõnad moodustavad ühe terviku)
#
# Kui sõna on kääne, siis ei lisandu lõppu täiendavat infot
# Kui sõna on verb, siis lisandub lõppu '.v'
# Kui sõna on osa erandlikust konstruktsioonist, kus käänet pole (siinsetes andmetes kaks juhtu), siis lisandub lõppu '.?'
# Muudel juhtudel lisandub lõppu '.adp', sest oletatavasti on tegemist kaassõnaga
#
# NB! Tegemist on esmase töötlusega, edaspidi muudetakse mustrite esitusviisi, aga mugavusest tehakse seda esmase töötluse
#                                                                                                         tulemuse baasilt

def process_len1(pattern_str):
    pieces = pattern_str.split()
    member_dict = {'wrd_1': '', 'wrd_2': '', 'wrd_3': ''}
    has_no_case = True
    n_cases = 0
    
    for i in range(len(pieces)):
        member_dict[f'wrd_{i+1}'] = pieces[i]
        if pieces[i] in case_question_mapping.keys():
            has_no_case = False
            n_cases+=1
            
    for key in member_dict.keys():
        if member_dict[key] == '' or member_dict[key] in case_question_mapping.keys():
            continue
        else:
            if has_no_case:
                member_dict[key] = member_dict[key]+'.?'
            elif 'V' in Text(member_dict[key]).tag_layer('morph_analysis').morph_analysis.partofspeech[0]:
                member_dict[key] = member_dict[key]+'.v'
            else:
                member_dict[key] = member_dict[key]+'.adp'
    
    return member_dict, n_cases

In [249]:
for idx, row in koopia.iterrows():
    if '+' not in row['government_with_case']:
        #print(row['government_with_case'])
        print(process_len1(row['government_with_case']))
        #print()
        break

({'wrd_1': 'part', 'wrd_2': '', 'wrd_3': ''}, 1)


In [250]:
wrd_1 = []
wrd_2 = []
wrd_3 = []
multiple_members = [] # ridade indeksid, kus on mitmeliikmeline (ehk kaheliikmeline) muster

new_df1 = koopia.copy()

for idx, row in koopia.iterrows():
    if '+' in row['government_with_case']:
        multiple_members.append(idx)
        new_df1 = new_df1.drop(idx)
    else:
        member_dict, n_cases = process_len1(row['government_with_case'])
        if n_cases > 1: # not len 1
            multiple_members.append(idx)
            new_df1 = new_df1.drop(idx)
        else:
            wrd_1.append(member_dict['wrd_1'])
            wrd_2.append(member_dict['wrd_2'])
            wrd_3.append(member_dict['wrd_3'])
        
new_df1['word_1'] = wrd_1
new_df1['word_2'] = wrd_2
new_df1['word_3'] = wrd_3

new_df1 = new_df1.reset_index(drop=True)

In [251]:
new_df2 = koopia.loc[multiple_members]
new_df2 = new_df2.reset_index(drop=True)

In [252]:
new_df1

,word,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case,word_1,word_2,word_3
0,aasima,keda*,aasima,NaN,NaN,NaN,part,part,,
1,aasima,kelle kallal,aasima,NaN,NaN,NaN,gen kallal,gen,kallal.adp,
2,abielluma,kellega,abielluma,NaN,NaN,NaN,kom,kom,,
3,abikätt ulatama,kellele,ulatama,abikätt,NaN,NaN,all,all,,
4,abstraheeruma,millest/kellest,abstraheeruma,NaN,NaN,NaN,el,el,,
...,...,...,...,...,...,...,...,...,...,...
2521,ümmardama,keda*,ümmardama,NaN,NaN,NaN,part,part,,
2522,üritama,mida teha,üritama,NaN,NaN,NaN,part teha,part,teha.v,
2523,ütlema,kellele,ütlema,NaN,NaN,NaN,all,all,,
2524,ütlema,kelle/mille kohta,ütlema,NaN,NaN,NaN,gen kohta,gen,kohta.adp,


In [253]:
# esmases formaadis mustrite salvestamine
new_df1.to_csv('data/verb_pattern_data/verb_patterns_len1.csv', encoding='utf-8')

In [ ]:
df_len1 = pd.read_csv('data/verb_pattern_data/verb_patterns_len1.csv')

In [ ]:
# üheliikmeliste mustrite esitusviisi tabelis muudetakse esmase esitusviisi põhjal

len1_cases = []
len1_adp = []
len1_verb = []
len1_other = []

for idx, row in df_len1.iterrows():
    parts = [row['word_1'], row['word_2'], row['word_3']]
    case = []
    adp = []
    verb = []
    other = []
    for part in parts:
        if not pd.isna(part):
            if len(part.split('.')) < 2: # kui eristavat lõputähist pole, läheb sõna käände tulpa
                case.append(part)
            else:
                if part.split('.')[1] == 'v': # kui lõpp tähistab verbi, läheb sõna verbi tulpa
                    verb.append(part.split('.')[0])
                elif part.split('.')[1] == 'adp': # kui lõpp tähistab kaassõna, läheb sõna kaasõna tulpa
                    adp.append(part.split('.')[0])
                else:
                    other.append(part.split('.')[0]) # muudel juhtudel läheb sõna muu tulpa
    len1_cases.append(';'.join(case))
    len1_adp.append(';'.join(adp))
    len1_verb.append(';'.join(verb))
    len1_other.append(';'.join(other))

df_len1['case'] = len1_cases
df_len1['adp'] = len1_adp
df_len1['verb'] = len1_verb
df_len1['other'] = len1_other

In [ ]:
df_len1_new = df_len1.drop(columns=['Unnamed: 0', 'word_1', 'word_2', 'word_3'])

In [ ]:
df_len1_new = df_len1_new.fillna('')

In [ ]:
# uuel kujul üheliikmeliste mustrite salvestamine
df_len1_new.to_csv('data/verb_pattern_data/verb_patterns_len1_new.csv', encoding='utf-8')

In [266]:
# kaheliikmeliste mustrite vähesuse tõttu tehakse mustrite osiste eraldustöö käsitsi excelis
new_df2.to_csv('data/verb_pattern_data/verb_patterns_len2_new.csv', encoding='utf-8')

### Mustrid andmebaasi

Failis *verb_patterns_new.db* on kaks tabelit : patterns_len1 (üheliikmelised mustrid) ja patterns_len2 (kaheliikmelised mustrid). Tabelis patterns_len1 on järgnevad veerud:

    ID - mustri ID
    word - verb või terviklik verbikonstruktsioon
    government - verbiga seotud fraas
    verb_word - verb/verbikonstruktsiooni peaverb
    compound_prt1 - pikema verbikonstruktsiooni korral esimene alluv
    compound_prt2 - pikema verbikonstruktsiooni korral teine alluv
    compound_prt3 - pikema verbikonstruktsiooni korral kolmas alluv
    full_pattern - verbiga seotud fraas, kus küsisõnad on teisendatud kääneteks
    w_case - verbiga seotud fraasi peasõna kääne
    adp - verbiga seotud fraasis olev kaassõna
    verb - verbiga seotud fraasis olev infiniitverb
    other - verbiga seotud fraasis olev liige, mis ei sobitu eelnimetatud kategooriatesse
    
Tabelis patterns_len2 on järgnevad veerud:

    ID - mustri ID
    word - verb või terviklik verbikonstruktsioon
    government - verbiga seotud fraasid
    verb_word - verb/verbikonstruktsiooni peaverb
    compound_prt1 - pikema verbikonstruktsiooni korral esimene alluv
    compound_prt2 - pikema verbikonstruktsiooni korral teine alluv
    compound_prt3 - pikema verbikonstruktsiooni korral kolmas alluv
    full_pattern - verbiga seotud fraasid, kus küsisõnad on teisendatud kääneteks
    member1_case - verbiga seotud esimese fraasi peasõna kääne
    member1_adp - verbiga seotud esimeses fraasis olev kaassõna
    member1_other - verbiga seotud esimeses fraasis olev liige, mis ei sobitu eelnimetatud kategooriatesse
    member2_case - verbiga seotud teise fraasi peasõna kääne
    member2_adp - verbiga seotud teises fraasis olev kaassõna
    member2_verb - verbiga seotud teises fraasis olev infiniitverb
    member2_other - verbiga seotud teises fraasis olev liige, mis ei sobitu eelnimetatud kategooriatesse

In [5]:
df_len1_new = pd.read_csv('data/verb_pattern_data/verb_patterns_len1_new.csv')
df_len2_new = pd.read_csv('data/verb_pattern_data/verb_patterns_len2_new.csv')

In [6]:
# verbimustrite andmebaasi loomine
con = sqlite3.connect("verb_patterns_new.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')

In [7]:
df_len1_new = df_len1_new.fillna('')
df_len2_new = df_len2_new.fillna('')

In [ ]:
new_df1_new = df_len1_new.drop(columns=['Unnamed: 0'])
new_df2_new = df_len2_new.drop(columns=['Unnamed: 0'])

In [8]:
# üheliikmelised mustrid
df_len1_new

,Unnamed: 0,word,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case,case,adp,verb,other
0,0,aasima,keda*,aasima,,,,part,part,,,
1,1,aasima,kelle kallal,aasima,,,,gen kallal,gen,kallal,,
2,2,abielluma,kellega,abielluma,,,,kom,kom,,,
3,3,abikätt ulatama,kellele,ulatama,abikätt,,,all,all,,,
4,4,abstraheeruma,millest/kellest,abstraheeruma,,,,el,el,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
2521,2521,ümmardama,keda*,ümmardama,,,,part,part,,,
2522,2522,üritama,mida teha,üritama,,,,part teha,part,,teha,
2523,2523,ütlema,kellele,ütlema,,,,all,all,,,
2524,2524,ütlema,kelle/mille kohta,ütlema,,,,gen kohta,gen,kohta,,


In [12]:
# kaheliikmelised mustrid
new_df2_new

,word,government,verb_word,compound_prt1,compound_prt2,compound_prt3,government_with_case,member1_case,member1_adp,member1_other,member2_case,member2_adp,member2_verb,member2_other
0,abistama,keda/mida* + milles,abistama,,,,part + in,part,,,in,,,
1,aitama,kellel + mida teha,aitama,,,,ad + part teha,ad,,,part,,teha,
2,alla kirjutama,mille/mida,kirjutama,alla,,,part gen,part,,,gen,,,
3,ette heitma,kellele/millele + mida*,heitma,ette,,,all + part,all,,,part,,,
4,informeerima,keda* + millest,informeerima,,,,part + el,part,,,el,,,
5,jagama,mida* + kellega,jagama,,,,part + kom,part,,,kom,,,
6,kaasa lööma,milles/kus,lööma,kaasa,,,in ad,in,,,ad,,,
7,kaasa tegema,milles/kus,tegema,kaasa,,,in ad,in,,,ad,,,
8,kannatama,mille all,kannatama,,,,gen all,gen,,,all,,,
9,keelama,kellel + mida teha,keelama,,,,ad + part teha,ad,,,part,,teha,


In [13]:
# andmebaasi tabelite loomine
cur.execute("CREATE TABLE patterns_len1(ID INTEGER PRIMARY KEY, word TEXT, government TEXT, verb_word TEXT, compound_prt1 TEXT, compound_prt2 TEXT, compound_prt3 TEXT, full_pattern TEXT, w_case TEXT, adp TEXT, verb TEXT, other TEXT)")
cur.execute("CREATE TABLE patterns_len2(ID INTEGER PRIMARY KEY, word TEXT, government TEXT, verb_word TEXT, compound_prt1 TEXT, compound_prt2 TEXT, compound_prt3 TEXT, full_pattern TEXT, member1_case TEXT, member1_adp TEXT, member1_other TEXT, member2_case TEXT, member2_adp TEXT, member2_verb TEXT, member2_other TEXT)")

In [14]:
# üheliikmeliste mustrite salvestamine andmebaasi
for idx, row in new_df1_new.iterrows():
    cur.execute("""INSERT INTO patterns_len1
                          (word, government, verb_word, compound_prt1, compound_prt2, compound_prt3, full_pattern, w_case, adp, verb, other) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);""", (row['word'], row['government'], row['verb_word'], row['compound_prt1'], row['compound_prt2'], row['compound_prt3'], row['government_with_case'], row['case'], row['adp'], row['verb'], row['other']))
    con.commit()

In [15]:
# kaheliikmeliste mustrite salvestamine andmebaasi
for idx, row in new_df2_new.iterrows():
    cur.execute("""INSERT INTO patterns_len2
                          (word, government, verb_word, compound_prt1, compound_prt2, compound_prt3, full_pattern, member1_case, member1_adp, member1_other, member2_case, member2_adp, member2_verb, member2_other) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);""", (row['word'], row['government'], row['verb_word'], row['compound_prt1'], row['compound_prt2'], row['compound_prt3'], row['government_with_case'], row['member1_case'], row['member1_adp'], row['member1_other'], row['member2_case'], row['member2_adp'], row['member2_verb'], row['member2_other']))
    con.commit()

In [16]:
con.close()